In [37]:
import cv2, torch
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from utils
from utils.eye_det_model import EyeDetModel

In [14]:
data = EyeDataModule(train_dataset_desc=pd.read_csv('train_eye_shape_dataset.csv'), val_dataset_desc=pd.read_csv('test_eye_shape_dataset.csv'), batch_size=1, n_workers=0)

In [16]:
model = Model.load_from_checkpoint('epoch=49-step=3900.ckpt')

In [17]:
trainer = pl.Trainer()
pred = trainer.predict(model, datamodule=data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\Workspace\Python\drowsy_detection\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [18]:
pred[0:5]

[tensor([[-8.5532,  8.2946]]),
 tensor([[-9.4860,  8.5556]]),
 tensor([[-4.4656,  4.9582]]),
 tensor([[-4.0379,  4.2877]]),
 tensor([[ 1.0945, -0.6631]])]

In [33]:
pred_result = list()

for pr in pred:
    pred_result.append(1-int(np.argmax(pr)))



In [34]:
pred_result[0:5]

[0, 0, 0, 0, 1]

In [35]:
val_data = pd.read_csv('test_eye_shape_dataset.csv')
val_data

,file_name,face_pt1_x_pos,face_pt1_y_pos,face_pt2_x_pos,face_pt2_y_pos,left_eye_pt1_x_pos,left_eye_pt1_y_pos,left_eye_pt2_x_pos,left_eye_pt2_y_pos,right_eye_pt1_x_pos,...,target_right_eye_pt1_y_pos,target_right_eye_pt2_x,target_right_eye_pt2_y_pos,exist_mask,exist_glasses,face_visible,left_eye_visible,left_eye_opened,right_eye_visible,right_eye_opened
0,d:\drowsy_dataset\image\bus\R_231_50_M\R_231_50_M_09_M1_G0_C0_03.jpg,226.0,505.0,538.0,964.0,319.0,632.0,338.0,657.0,409.0,...,659.53,478.52,693.83,1,0,1,1,1,1,1
1,d:\drowsy_dataset\image\bus\R_237_40_M\R_237_40_M_20_M0_G0_C0_05.jpg,151.0,246.0,470.0,762.0,197.0,431.0,255.0,450.0,347.0,...,414.25,419.07,451.02,0,0,1,1,1,1,1
2,d:\drowsy_dataset\image\bus\R_221_50_M\R_221_50_M_04_M1_G0_C0_15.jpg,151.0,522.0,416.0,972.0,245.0,636.0,277.0,656.0,330.0,...,677.20,385.39,711.52,1,0,1,1,0,1,1
3,d:\drowsy_dataset\image\bus\R_221_50_M\R_221_50_M_12_M1_G0_C0_06.jpg,244.0,540.0,504.0,968.0,349.0,628.0,383.0,652.0,363.0,...,716.67,491.77,763.04,1,0,1,1,1,1,1
4,d:\drowsy_dataset\image\bus\R_221_50_M\R_221_50_M_13_M1_G0_C0_02.jpg,190.0,571.0,464.0,981.0,304.0,640.0,320.0,667.0,278.0,...,732.69,443.64,772.16,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193,d:\drowsy_dataset\image\truck\R_623_40_M\R_623_40_M_15_M0_G1_C0_02.jpg,265.0,317.0,566.0,786.0,356.0,486.0,408.0,499.0,484.0,...,485.00,547.00,523.00,0,1,1,1,1,1,1
2194,d:\drowsy_dataset\image\truck\R_632_50_M\R_632_50_M_19_M0_G1_C0_01.jpg,215.0,412.0,537.0,923.0,283.0,569.0,340.0,585.0,430.0,...,566.00,498.00,611.00,0,1,1,1,0,1,1
2195,d:\drowsy_dataset\image\truck\R_614_20_M\R_614_20_M_10_M0_G1_C0_15.jpg,260.0,435.0,560.0,852.0,297.0,561.0,353.0,577.0,445.0,...,535.00,511.00,572.00,0,1,1,1,1,1,1
2196,d:\drowsy_dataset\image\truck\R_614_20_M\R_614_20_M_06_M0_G1_C0_07.jpg,288.0,411.0,579.0,847.0,346.0,547.0,399.0,560.0,487.0,...,541.00,550.00,578.00,0,1,1,1,1,1,1


In [38]:
confusion_matrix(1-val_data['left_eye_opened'], pred_result)

array([[1866,   71],
       [ 108,  153]], dtype=int64)

In [42]:
accuracy_score(1-val_data['left_eye_opened'], pred_result)

0.9185623293903549

In [39]:
precision_score(1-val_data['left_eye_opened'], pred_result)

0.6830357142857143

In [40]:
recall_score(1-val_data['left_eye_opened'], pred_result)

0.5862068965517241

In [41]:
f1_score(1-val_data['left_eye_opened'], pred_result)

0.6309278350515464